# Michigan COVID-19 Data

## Imports

In [ ]:
import pandas as pd
import numpy as np

import os
import glob
import requests
from bs4 import BeautifulSoup

import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gp

## Michigan COVID-19 Data
### Download the latest COVID-19 data

In [ ]:
michiganCovid19Url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html'
michiganCovid19PageSoup = BeautifulSoup(requests.get(michiganCovid19Url).content)

Make the michigan external data directory to store the data if it isn't there already. 

In [ ]:
externalPath = '../data/external/michiganCovid19Data/'
dir = os.path.dirname(externalPath)
if not os.path.exists(dir):
    os.makedirs(dir)

Download the data into the the above folder.

In [ ]:
# Delete the existing data as michigan datetime stamps their data
michiganDataFileNames = glob.glob('../data/external/michiganCovid19Data/*')
for fileName in michiganDataFileNames:
    os.remove(fileName)

In [ ]:
for urlHtml in michiganCovid19PageSoup.find(id='comp_115341').find_all('a'):
    url = 'https://www.michigan.gov/' + urlHtml['href']
    with open(externalPath + url.split('/')[-1],"wb") as file:
        response = requests.get(url)
        file.write(response.content)

### Read the day by day Michigan covid 19 data

In [ ]:
glob.glob('../data/external/michiganCovid19Data/Cases_and_Deaths_by_County_and_by_Date_of_Symptom_Onset_or_by_Date_of_Death*')[0]
michiganDayByDayDf = pd.read_excel(glob.glob('../data/external/michiganCovid19Data/Cases_and_Deaths_by_County_and_by_Date_of_Symptom_Onset_or_by_Date_of_Death*')[0])
#df = pd.read_excel("../data/external/michiganCovid19Data/Cases_and_Deaths_by_County_2020-11-02_706751_7.xlsx")
#df = pd.read_excel("../data/external/michiganCovid19Data/Cases_by_Demographics_Statewide_2020-11-02_706753_7.xlsx")
#df = pd.read_excel("../data/external/michiganCovid19Data/Covid-19_Tests_by_County_2020-11-02_706754_7.xlsx")
#df = pd.read_excel("../data/external/michiganCovid19Data/Diagnostic_Tests_by_Result_and_County_2020-11-02_706755_7.xlsx")
michiganDayByDayTestingDf = pd.read_excel(glob.glob('../data/external/michiganCovid19Data/Diagnostic_Tests_by_Result_and_County*')[0])

In [ ]:
michiganDayByDayDf

### Tidy data
Let's look to see if there is any missing data. Looks like the date data is missing.

In [ ]:
michiganDayByDayDf.count()/michiganDayByDayDf.shape[0]

Since only 3 dates are missing and they don't seem impactful, let's filter out the missing date values.

In [ ]:
michiganDayByDayDf[~np.isnat(michiganDayByDayDf["Date"])]

Let's look at the datatypes. Wow, it got most of them right! Let's get the CASE_STATUS and COUNTY set as category.

In [ ]:
michiganDayByDayDf.dtypes

In [ ]:
michiganDayByDayDf['CASE_STATUS'] = michiganDayByDayDf['CASE_STATUS'].astype('category')
michiganDayByDayDf['COUNTY'] = michiganDayByDayDf['COUNTY'].astype('category')

In [ ]:
michiganDayByDayDf.dtypes

In [ ]:
michiganDayByDayDf = michiganDayByDayDf[michiganDayByDayDf['CASE_STATUS'] == 'Confirmed']
michiganDayByDayDf

### Let's graphically view the data

Is there a correlation between cases and deaths?

In [ ]:
dailyCaseDeathMichigan = michiganDayByDayDf[["Date","Cases","Deaths"]].groupby(["Date"]).sum()

In [ ]:
sns.pairplot(dailyCaseDeathMichigan)

# Get Michigan GeoJSON

In [ ]:
michiganCountyGeoJSONURL = "https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/MI-26-michigan-counties.json"

Citation:

Eldersveld, D. (2020) TopoJSON Collection (Version 1.0) [Source Code]. https://github.com/deldersveld/topojson.

## Plot empty map to confirm data source fitness

In [ ]:
geoDataFrame = gp.read_file(michiganCountyGeoJSONURL)
geoDataFrame.plot()

## Understanding the GeoJSON Data

In [ ]:
geoDataFrame.head(5)

## Create a county organized Covid-19 DataFrame

In [ ]:
countyDataFrame = michiganDayByDayDf.groupby(['COUNTY']).sum()

## Merge GeoJSON with Covid-19 DataFrame

In [ ]:
geoDataFrameMerged = geoDataFrame.merge(countyDataFrame, right_on="COUNTY", left_on="NAME")

In [ ]:
geoDataFrameMerged.head(5)

## Modify the DataFrame for centering labels for ease of comprehension.

In [ ]:
geoDataFrameMerged["center"] = geoDataFrameMerged["geometry"].centroid
michiganCountyNames = geoDataFrameMerged.copy()
michiganCountyNames.set_geometry("center", inplace=True)

## Plot Case County Data

In [ ]:
ax = geoDataFrameMerged.plot(column="Cases", legend=True, figsize=(15,10), cmap='YlOrRd')

for x, y, label in zip(michiganCountyNames.geometry.x, michiganCountyNames.geometry.y, michiganCountyNames["NAME"]):
    plt.text(x, y, label, fontsize = 10)

## Plot Death County Data

In [ ]:
ax = geoDataFrameMerged.plot(column="Deaths", legend=True, figsize=(15,10), cmap='YlOrRd')

for x, y, label in zip(michiganCountyNames.geometry.x, michiganCountyNames.geometry.y, michiganCountyNames["NAME"]):
    plt.text(x, y, label, fontsize = 10)

# Export Data for Final Project Notebook
## First export the data by county

In [ ]:
michiganDayByDayDf["State"] = 'Michigan'

In [ ]:
michiganExportDataFrameByCounty = michiganDayByDayDf[["Date","State","COUNTY","Cases","Deaths"]]

In [ ]:
michiganExportDataFrameByCounty.columns=["Date","State","County","Cases","Deaths"]

In [ ]:
michiganExportDataFrameByCounty

In [ ]:
michiganExportDataFrameByCounty.to_csv("../data/processed/by_county/michigan.csv", index=False)

## Second export by state

In [ ]:
michiganDayByDayTestingDf.columns=["County","Date", "Negative", "Positive","Total"]
michiganDayByDayTestingDf.groupby(["Date"]).sum()

In [ ]:
michiganExportDataFrameByState = michiganExportDataFrameByCounty.groupby("Date").sum()

In [ ]:
michiganExportDataFrameByState["State"] = "Michigan"

In [ ]:
michiganExportDataFrameByState

In [ ]:
groupedTestingByDate = michiganDayByDayTestingDf.groupby("Date").sum()

In [ ]:
groupedTestingByDate

In [ ]:
michiganExportDataFrameByState = michiganExportDataFrameByState.merge(groupedTestingByDate,left_index=True,right_index=True)

In [ ]:
michiganExportDataFrameByState = michiganExportDataFrameByState[["State","Cases","Positive","Negative","Deaths"]]

In [ ]:
michiganExportDataFrameByState.to_csv("../data/processed/by_state/michigan.csv")